In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import torch
import math

from modules.util.util import importstr
from modules.util.logconf import logging
log = logging.getLogger('nb')

In [3]:
def run(app, *argv):    
    torch.cuda.empty_cache()
    
    argv = list(argv)
    argv.insert(0, '--num-workers=4')
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}.main()".format(app, argv))

In [4]:
# width_irc=(15,192,192)
# run('prepcache.CovidPrepCacheApp', '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}')

2020-11-26 01:10:15,090 INFO     pid:7031 nb:006:run Running: prepcache.CovidPrepCacheApp(['--num-workers=4', '--width-irc', '15', '192', '192']).main()
2020-11-26 01:10:30,296 INFO     pid:7031 prepcache:039:main Starting CovidPrepCacheApp, Namespace(batch_size=1024, num_workers=4, width_irc=['15', '192', '192'])
100%|██████████| 10/10 [04:55<00:00, 29.60s/it]
2020-11-26 01:15:26,419 INFO     pid:7031 nb:011:run Finished: prepcache.CovidPrepCacheApp.['--num-workers=4', '--width-irc', '15', '192', '192'].main()


## Test Run

In [4]:
batch_size = 2
epochs = 3
steps_per_epoch=200
val_cadence = 5
recall_priority = 0
unet_depth=3
pad_types = ['zero', 'replicate']
windows = [None, 'lung', 'mediastinal', 'dist']
project_name = 'covid19_seg'
width_irc = (12,192,192)

Cache all the different sizes of bounding box

In [5]:
run_name = 'focal tversky loss' #, dice_loss, rand_coords, shifted_lung_window'

In [1]:
run('training.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', 
    f'--steps-per-epoch={steps_per_epoch}', f'--val-cadence={val_cadence}', f'--recall-priority={recall_priority}',
    f'--depth={unet_depth}', f'--pad-type={pad_types[1]}', '--augmented', f'--model-path={model_path}',
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', 
    f'--project-name={project_name}', f'--run-name={run_name}')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

## Long Run

In [4]:
epochs = 500
batch_size = 32
unet_depth=3
val_cadence = 5
recall_priority = 0
pad_type = 'reflect'
window = 'shifted_lung'
steps_per_epoch = 100000
width_irc=(15,192,192)
project_name = 'covid19_seg'
loss_func = 'dice_loss'

In [5]:
run_name = 'b192 w15 train run'

In [1]:
run('training.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', 
    f'--ct-window={window}', '--augmented', f'--val-cadence={val_cadence}', f'--depth={unet_depth}', 
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', f'--pad-type={pad_type}', 
    f'--steps-per-epoch={steps_per_epoch}', f'--project-name={project_name}', f'--run-name={run_name}')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')